<a href="https://colab.research.google.com/github/Akmadiha27/Building-LLMS-From-scratch/blob/main/Tokenizer_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Reading raw text from the file

In [1]:
with open ("/content/the-verdict (1).txt","r",encoding="utf-8") as f:
  raw_txt=f.read()
print("The number of characters are:", len(raw_txt))
print(raw_txt[:99])

The number of characters are: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


Split the text to make tokens and remove whitespaces also!!
Whitespace -> reduces memory but if code sensible to structure then we need to keep it.
Question marks, quotations we need all as seperate tokens

In [2]:
import re


In [3]:
preprocessed=re.split(r'([,.:;?_!"()\']|--|\s)',raw_txt)
preprocessed=[item.strip() for item in preprocessed if item.strip()]
print(preprocessed[:40])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in', 'the', 'height', 'of', 'his', 'glory', ',', 'he', 'had', 'dropped', 'his']


In [4]:
print(len(preprocessed))

4690


Converting tokens into token ids so that they have numerical representation

First we will se how many words are there

In [5]:
  all_words=sorted(set(preprocessed))
  print(len(all_words))

1130


Har ek token ko ek number dere ab yahan

In [6]:
vocab={token:integer for integer,token in enumerate(all_words)}

In [7]:
for i,item in enumerate(vocab.items()):
  print(item)
  if i>=20:
    break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)


Reverse dictionary to be made so that we have word from id

## **SIMPLE TOKENIZER CLASS IN PYTHON**

In [8]:
class SimpleTokenizerV1:
  def __init__(self,vocab):
    self.str_to_int=vocab
    self.int_to_str={i:s for s,i in vocab.items()}
  def encode(self,text):
    preprocessed=re.split(r'([,.:;?_!"()\']|--|\s)',text)
    preprocessed=[item.strip() for item in preprocessed if item.strip()]
    ids=[self.str_to_int[s] for s in preprocessed]
    return ids
  def decode(self,ids):
    text=" ".join([self.int_to_str[i]for i in ids])
    #replacing spaces before specified punctuations
    text=re.sub(r'\s+([,.?!"()\'])',r'\1',text)
    return text

In [9]:
tokenizer = SimpleTokenizerV1(vocab)

text = """"It's the last he painted, you know,"
           Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
print(ids)

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


converting back to original set

In [10]:
tokenizer.decode(ids)

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

*Note that if particular text is not there in the vocab then it gives error*

## **ADDING SPECIAL CONTEXT TOKENS**

In [22]:
all_tokens=sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>","<|unk|>"])

vocab={token:integer for integer,token in enumerate(all_tokens)}

In [23]:
len(vocab.items())

1132

In [24]:
for i,item in enumerate(list(vocab.items())[-5:]):
  print(item)

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


In [25]:
class SimpleTokenizerV2:
  def __init__(self,vocab):
    self.str_to_int=vocab
    self.int_to_str={i:s for s,i in vocab.items()}
  def encode(self,text):
    preprocessed=re.split(r'([,.:;?_!"()\']|--|\s)',text)
    preprocessed=[item.strip() for item in preprocessed if item.strip()]
    #new thing is add unknown
    preprocessed=[item if item in self.str_to_int else "<|unk|>" for item in preprocessed]
    ids=[self.str_to_int[s] for s in preprocessed]
    return ids
  def decode(self,ids):
    text=" ".join([self.int_to_str[i]for i in ids])
    #replacing spaces before specified punctuations
    text=re.sub(r'\s+([,.?!"()\'])',r'\1',text)
    return text

In [26]:
tokenizer=SimpleTokenizerV2(vocab)
text1="Hello, do you like tea?"
text2="In the sunlit terraces of the palace."

text="<|endoftext|>".join((text1,text2))
print(text)

Hello, do you like tea?<|endoftext|>In the sunlit terraces of the palace.


In [27]:
tokenizer.encode(text)

[1131, 5, 355, 1126, 628, 975, 10, 1131, 988, 956, 984, 722, 988, 1131, 7]

In [28]:
tokenizer.decode(tokenizer.encode(text))

'<|unk|>, do you like tea? <|unk|> the sunlit terraces of the <|unk|>.'

we also have other tokens like BOS,EOS,PAD

GPT uses only endoftext tokens | unk is also not used, it does byte pair encoding.